# Statistics on SVM

We will arange data and train SVC on the data.
Then we will try to train again only using quarters from the same cluster (using the quarter classifier).

Our randomly chosen 5 stocks are: 
'SHW', 'MNK', 'BIO', 'KYTH', 'KRO'


In [1]:
from Utilities.orginizers import LearningData, TrainingData
from sklearn.model_selection import KFold, cross_val_score
from itertools import product

from multiprocessing import Pool
import matplotlib.pyplot as plt
import Utilities
import numpy as np
import warnings
import logging
import pandas

%matplotlib inline

ld = LearningData()

In [2]:
stock_names = ('SHW', 'MNK', 'BIO', 'KYTH', 'KRO')
logging.getLogger().setLevel(logging.INFO)
stocks = [TrainingData(sn, ld=ld).add_history(10).set_threshold(0.8).get() for sn in stock_names]

INFO:root:TrainingData: name=SHW, days_forward=1: threshold found is 0
INFO:root:TrainingData: name=SHW, days_forward=1: threshold found is 0.018134653877025918
INFO:root:TrainingData: name=MNK, days_forward=1: threshold found is 0
INFO:root:TrainingData: name=MNK, days_forward=1: threshold found is 0.026397640541565882
INFO:root:TrainingData: name=BIO, days_forward=1: threshold found is 0
INFO:root:TrainingData: name=BIO, days_forward=1: threshold found is 0.018548151336335694
INFO:root:TrainingData: name=KYTH, days_forward=1: threshold found is 0
INFO:root:TrainingData: name=KYTH, days_forward=1: threshold found is 0.036464497748678756
INFO:root:TrainingData: name=KRO, days_forward=1: threshold found is 0
INFO:root:TrainingData: name=KRO, days_forward=1: threshold found is 0.0321798749747933


In [3]:
import os
print("Samples per stock:")
print(os.linesep.join(['{} {}'.format(sn, len(data)) for sn, (data, classes) in zip(stock_names, stocks)]))
print("percent of positive classification per stock")
print(os.linesep.join(['{} {}'.format(sn, float(classes.sum())/len(classes)) for sn, (data, classes) in zip(stock_names, stocks)]))
print(stocks[0][0].describe())

Samples per stock:
SHW 7855
MNK 819
BIO 2815
KYTH 718
KRO 3220
percent of positive classification per stock
SHW 0.20038192234245703
MNK 0.1978021978021978
BIO 0.15381882770870337
KYTH 0.201949860724234
KRO 0.20031055900621117
             change        close   ex-divident         high           low  \
count  7.855000e+03  7855.000000  7.855000e+03  7855.000000  7.855000e+03   
mean   2.170977e-17     0.000000  1.447318e-17     0.000000 -1.447318e-16   
std    1.000064e+00     1.000064  1.000064e+00     1.000064  1.000064e+00   
min   -9.826291e-01    -0.711964 -8.285474e-02    -0.708525 -7.122042e-01   
25%   -6.546142e-01    -0.545740 -8.285474e-02    -0.546135 -5.453623e-01   
50%   -2.623907e-01    -0.438244 -8.285474e-02    -0.440260 -4.366827e-01   
75%    3.430775e-01    -0.053682 -8.285474e-02    -0.052620 -5.562959e-02   
max    1.404981e+01     3.669051  2.486545e+01     3.637710  3.656416e+00   

               open   split_ratio        volume  change1_days_before  \
count  7

In [4]:
def run_model(model, stocks=stocks):
    scores = {sn: [] for sn in stock_names}
    validation_rounds = 3
    for i in range(validation_rounds):
        for sn, (data, classes) in zip(stock_names, stocks):
            scores[sn].append(sum(cross_val_score(model, data, classes))/3) 
    for sn in scores.keys():
        scores[sn] = sum(scores[sn])/validation_rounds
    return scores

def get_best_model(models, accs):
    best_model = (0 , models[0])
    for model, accs in zip(models, accs):
        if sum(accs.values()) > best_model[0]:
            best_model = (sum(accs.values()), model)
    return best_model

def median(lst):
    quotient, remainder = divmod(len(lst), 2)
    if remainder:
        return sorted(lst)[quotient]
    return sum(sorted(lst)[quotient - 1:quotient + 1]) / 2.

def get_best_median_model(models, accs):
    best_model = (0 , models[0])
    for model, accs in zip(models, accs):
        if median(accs.values()) > best_model[0]:
            best_model = (median(accs.values()), model)
    return best_model

In [9]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
import traceback

# create iterables for all options and use itertools product
C=[0.1, 0.5, 0.8, 1.2]
n_estimators = [10,50,100,500]
learning_rate = [0.5, 1.0, 1.5]


asmodels = [AdaBoostClassifier(SVC(C=c, max_iter=1200,probability=True), n_estimators=ne, learning_rate=lr)
          for c, ne, lr in product(C, n_estimators, learning_rate)]

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    asaccuracies = list(map(run_model, asmodels))

In [ ]:
from pandas import DataFrame

# data should be list of lists and each row should be:
# criterion, max_depth, min_samples, stock, accuracy
data = []
for model, accs in zip(asmodels, asaccuracies):
    for sn, acc in accs.items():
        data.append((model.C, model.kernel, model.max_iter, sn, acc))

asresults = DataFrame(data, columns=('C', 'n_estimators', 'learning_rate', 'stock', 'accuracy'))

In [ ]:
plt.rc('font', size=30)
fig = plt.figure(figsize=(30, 20))
alpha = 0.6

ax1 = plt.subplot2grid((1,1), (0,0))
for name, group in asresults.groupby('stock'):
    # since we are counting values it makes sense to regularize the counts
    group.groupby('C').agg('mean')['accuracy'].plot(kind='line', label=name, alpha=alpha)
ax1.set_xlabel('C avlue to accuracy')
plt.legend(loc='best')
ax1.set_title("What's the best C value?" )

In [ ]:
plt.rc('font', size=30)
fig = plt.figure(figsize=(30, 20))
alpha = 0.6

ax1 = plt.subplot2grid((1,1), (0,0))
for name, group in asresults.groupby('stock'):
    # since we are counting values it makes sense to regularize the counts
    group.groupby('n_estimators').agg('mean')['accuracy'].plot(kind='line', label=name, alpha=alpha)
ax1.set_xlabel('n_estimators to accuracy')
plt.legend(loc='best')
ax1.set_title("What's the best n_estimators?" )

In [ ]:
plt.rc('font', size=30)
fig = plt.figure(figsize=(30, 20))
alpha = 0.6

ax1 = plt.subplot2grid((1,1), (0,0))
for name, group in asresults.groupby('stock'):
    # since we are counting values it makes sense to regularize the counts
    group.groupby('learning_rate').agg('mean')['accuracy'].plot(kind='line', label=name, alpha=alpha)
ax1.set_xlabel('learning_rate to accuracy')
plt.legend(loc='best')
ax1.set_title("What's the best learning_rate?" )

The best average accuracy model is:

In [ ]:
print(get_best_model(svm_models, saccuracies))
print(get_best_median_model(svm_models, saccuracies))

Which is mostly expected.
Also seems to be the best C value is not hit yet. The problem is that low C ignores mistakes. so we will check if we can have good classification with higher C and more iterations. We will try again with less parameters, medium C values and more iterations.
Let us check:

In [ ]:
# create iterables for all options and use itertools product
C=[0.8, 1.0 ,1.5]
max_iter=[1000, 1200, 1500, 2000]

s2models = [SVC(C=c, max_iter=mi) 
          for c, mi in product(C, max_iter)]

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    s2accuracies = list(map(run_model, s2models))

In [ ]:
from pandas import DataFrame

# data should be list of lists and each row should be:
# criterion, max_depth, min_samples, stock, accuracy
data = []
for model, accs in zip(s2models, s2accuracies):
    for sn, acc in accs.items():
        data.append((model.C, model.max_iter, sn, acc))

s2results = DataFrame(data, columns=('C', 'max_iter', 'stock', 'accuracy'))

In [ ]:
plt.rc('font', size=30)
fig = plt.figure(figsize=(30, 20))
alpha = 0.6

ax1 = plt.subplot2grid((1,1), (0,0))
for name, group in s2results.groupby('stock'):
    # since we are counting values it makes sense to regularize the counts
    group.groupby('max_iter').agg('mean')['accuracy'].plot(kind='line', label=name, alpha=alpha)
ax1.set_xlabel('max_iter to accuracy')
plt.legend(loc='best')
ax1.set_title("What's the best max_iter?" )

In [ ]:
plt.rc('font', size=30)
fig = plt.figure(figsize=(30, 20))
alpha = 0.6

ax1 = plt.subplot2grid((1,1), (0,0))
for name, group in s2results.groupby('stock'):
    # since we are counting values it makes sense to regularize the counts
    group.groupby('C').agg('mean')['accuracy'].plot(kind='line', label=name, alpha=alpha)
ax1.set_xlabel('C avlue to accuracy')
plt.legend(loc='best')
ax1.set_title("What's the best C value?" )

From checking it seems truly a lower C value should be chosen from what we decide to limit by.
We will need to choose for a C value and kernel=rbf and max_iter>=1200. This gives much better results then trees and we don't need to coerce our data to catagories. And it seems it really influences as the accuracy of BIO dropped really hard because we dont allow svm to be the default classifier


In [ ]:
print(get_best_model(s2models, s2accuracies))
print(get_best_median_model(s2models, s2accuracies))